In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import laplacian
import hnswlib

# SPADE 所需的函数
def hnsw(features, k=10, ef=100, M=48):
    print("Initializing HNSW Index...")
    num_samples, dim = features.shape
    print(f"Number of samples: {num_samples}, Dimension: {dim}")
    p = hnswlib.Index(space='l2', dim=dim)
    p.init_index(max_elements=num_samples, ef_construction=ef, M=M)
    labels_index = np.arange(num_samples)
    p.add_items(features, labels_index)
    p.set_ef(ef)
    neighs, weight = p.knn_query(features, k + 1)
    print(f"Found nearest neighbors for k={k + 1}")
    return neighs, weight

def construct_adj(neighs, weight):
    print("Constructing adjacency matrix...")
    dim = neighs.shape[0]
    k = neighs.shape[1] - 1

    idx0 = np.arange(dim)
    row = np.repeat(idx0.reshape(-1,1), k, axis=1).reshape(-1,)
    col = neighs[:, 1:].reshape(-1,)
    all_row = np.concatenate((row, col), axis=0)
    all_col = np.concatenate((col, row), axis=0)
    data = np.ones(all_row.shape[0])
    adj = csr_matrix((data, (all_row, all_col)), shape=(dim, dim))
    print("Adjacency matrix constructed.")
    return adj

#  计算SPADE score
def spade_score(input_features, output_features, k=10, num_eigs=2):
    print("Calculating SPADE score...")
    # 构建输入和输出的邻接矩阵
    neighs_in, dist_in = hnsw(input_features, k)
    adj_in = construct_adj(neighs_in, dist_in)
    neighs_out, dist_out = hnsw(output_features, k)
    adj_out = construct_adj(neighs_out, dist_out)

    # 计算拉普拉斯矩阵
    print("Calculating Laplacian matrices...")
    L_in = laplacian(adj_in, normed=True)
    L_out = laplacian(adj_out, normed=True)

    # 计算广义特征值和特征向量
    print("Computing eigenvalues and eigenvectors...")
    eigvals_in, eigvecs_in = np.linalg.eig(L_in.toarray())
    eigvals_out, eigvecs_out = np.linalg.eig(L_out.toarray())

    # 选择最大特征值作为 SPADE 分数
    spade_score = max(eigvals_out) / max(eigvals_in)
    print(f"SPADE Score: {spade_score}")
    return spade_score

# 加载 ResNet50 模型
print("Loading ResNet50 model...")
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()

# 数据集和数据加载器（使用 CIFAR10 的 100 个数据样本）
print("Loading CIFAR10 dataset with 100 samples...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)
subset = Subset(dataset, list(range(100)))  
dataloader = DataLoader(subset, batch_size=100, shuffle=False)

input_features = []
output_features = []

print("Extracting input and output features...")
with torch.no_grad():
    for images, _ in dataloader:
        # 将图片展平
        input_features.append(images.view(images.size(0), -1).numpy())
        print("Extracted input features.")
        

        features = resnet50(images)
        output_features.append(features.numpy())
        print("Extracted output features.")

# 转换为 numpy 数组
input_features = np.vstack(input_features)
output_features = np.vstack(output_features)
print("Converted features to numpy arrays.")

# 计算 ResNet50 的 SPADE 分数
spade_score_resnet50 = spade_score(input_features, output_features)
print("ResNet50 的 SPADE 分数（鲁棒性指标）：", spade_score_resnet50)


Loading ResNet50 model...


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading CIFAR10 dataset with 100 samples...
Files already downloaded and verified
Extracting input and output features...
Extracted input features.
Extracted output features.
Converted features to numpy arrays.
Calculating SPADE score...
Initializing HNSW Index...
Number of samples: 100, Dimension: 150528
Found nearest neighbors for k=11
Constructing adjacency matrix...
Adjacency matrix constructed.
Initializing HNSW Index...
Number of samples: 100, Dimension: 1000
Found nearest neighbors for k=11
Constructing adjacency matrix...
Adjacency matrix constructed.
Calculating Laplacian matrices...
Computing eigenvalues and eigenvectors...
SPADE Score: 0.9639618102452412
ResNet50 的 SPADE 分数（鲁棒性指标）： 0.9639618102452412


In [6]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import laplacian
import hnswlib

# SPADE 所需的函数
def hnsw(features, k=10, ef=100, M=48):
    print("Initializing HNSW Index...")
    num_samples, dim = features.shape
    print(f"Number of samples: {num_samples}, Dimension: {dim}")
    p = hnswlib.Index(space='l2', dim=dim)
    p.init_index(max_elements=num_samples, ef_construction=ef, M=M)
    labels_index = np.arange(num_samples)
    p.add_items(features, labels_index)
    p.set_ef(ef)
    neighs, weight = p.knn_query(features, k + 1)
    print(f"Found nearest neighbors for k={k + 1}")
    return neighs, weight

def construct_adj(neighs, weight):
    print("Constructing adjacency matrix...")
    dim = neighs.shape[0]
    k = neighs.shape[1] - 1

    idx0 = np.arange(dim)
    row = np.repeat(idx0.reshape(-1,1), k, axis=1).reshape(-1,)
    col = neighs[:, 1:].reshape(-1,)
    all_row = np.concatenate((row, col), axis=0)
    all_col = np.concatenate((col, row), axis=0)
    data = np.ones(all_row.shape[0])
    adj = csr_matrix((data, (all_row, all_col)), shape=(dim, dim))
    print("Adjacency matrix constructed.")
    return adj

#  计算SPADE score
def spade_score(input_features, output_features, k=10, num_eigs=2):
    print("Calculating SPADE score...")
    # 构建输入和输出的邻接矩阵
    neighs_in, dist_in = hnsw(input_features, k)
    adj_in = construct_adj(neighs_in, dist_in)
    neighs_out, dist_out = hnsw(output_features, k)
    adj_out = construct_adj(neighs_out, dist_out)

    # 计算拉普拉斯矩阵
    print("Calculating Laplacian matrices...")
    L_in = laplacian(adj_in, normed=True)
    L_out = laplacian(adj_out, normed=True)

    # 计算广义特征值和特征向量
    print("Computing eigenvalues and eigenvectors...")
    eigvals_in, eigvecs_in = np.linalg.eig(L_in.toarray())
    eigvals_out, eigvecs_out = np.linalg.eig(L_out.toarray())

    # 选择最大特征值作为 SPADE 分数
    spade_score = max(eigvals_out) / max(eigvals_in)
    print(f"SPADE Score: {spade_score}")
    return spade_score

# 加载 ResNet50 模型
print("Loading ResNet50 model...")
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()

# 数据集和数据加载器（使用 CIFAR10 全部数据样本）
print("Loading CIFAR10 ALL dataset samples...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)
 
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

input_features = []
output_features = []

print("Extracting input and output features...")
with torch.no_grad():
    for images, _ in dataloader:
        # 将图片展平
        input_features.append(images.view(images.size(0), -1).numpy())
        print("Extracted input features.")
        

        features = resnet50(images)
        output_features.append(features.numpy())
        print("Extracted output features.")

# 转换为 numpy 数组
input_features = np.vstack(input_features)
output_features = np.vstack(output_features)
print("Converted features to numpy arrays.")

# 计算 ResNet50 的 SPADE 分数
spade_score_resnet50 = spade_score(input_features, output_features)
print("ResNet50 的 SPADE 分数（鲁棒性指标）：", spade_score_resnet50)


Loading ResNet50 model...
Loading CIFAR10 dataset with 100 samples...
Files already downloaded and verified
Extracting input and output features...
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.
Extracted output features.
Extracted input features.


KeyboardInterrupt: 